## Cargar el dataset
1. Cargar el dataset que son imagenes que están dentro de carpetas separadas por test y train y cada una separada por maligno y benigno.
2. Crearemos un dataset en el que por cada fila se incluye el path y si es benigno o maligno.
3. Tenemos 4000 imagenes para el train y 2000 para el test.

### Dataframe train
Obtener todas las imágenes dentro de la carpeta de train y guardar los Path junto con los nombres de las imagenes y si es maligna o benigna en un Dataframe.

In [133]:
import os
import pandas as pd

# Train
def train_df(train_path):
    classes = [] # Benignas o malignas
    class_paths = [] # Paths de las imagenes
    image_names = [] # Nombre de las imagenes

    # Obtener el contenido de la carpeta Train (Benigna y Maligna)
    files = os.listdir(train_path)
    for file in files: 
        label_dir = os.path.join(train_path, file)
        label = os.listdir(label_dir)

        # Obtener el contenido de la carpeta Benigna y Maligna (Imagenes)
        for image in label:
            if not image.startswith('.'): # Descartar ficheros ocultos
                image_names.append(image) 
                class_paths.append(label_dir) 
                classes.append(file) 

    # Crear el dataframe
    class_paths = pd.Series(class_paths, name='Class Path')
    image_names = pd.Series(image_names, name='Image name')
    image_classes = pd.Series(classes, name='Class') 
    tr_df = pd.concat([class_paths, image_names, image_classes], axis=1)
    
    return tr_df

tr_df = train_df('./skin-cancer-dataset/train')
print(tr_df)
groupped = tr_df.groupby(["Class"]).count()
print(groupped)

                                 Class Path Image name      Class
0     ./skin-cancer-dataset/train/Malignant     63.jpg  Malignant
1     ./skin-cancer-dataset/train/Malignant    823.jpg  Malignant
2     ./skin-cancer-dataset/train/Malignant   1409.jpg  Malignant
3     ./skin-cancer-dataset/train/Malignant    189.jpg  Malignant
4     ./skin-cancer-dataset/train/Malignant     77.jpg  Malignant
...                                     ...        ...        ...
3994     ./skin-cancer-dataset/train/Benign    190.jpg     Benign
3995     ./skin-cancer-dataset/train/Benign   1404.jpg     Benign
3996     ./skin-cancer-dataset/train/Benign   1410.jpg     Benign
3997     ./skin-cancer-dataset/train/Benign    184.jpg     Benign
3998     ./skin-cancer-dataset/train/Benign   1376.jpg     Benign

[3999 rows x 3 columns]
           Class Path  Image name
Class                            
Benign           2000        2000
Malignant        1999        1999


### Dataframe test
Obtener todas las imágenes dentro de la carpeta de test y guardar los Path junto con los nombres de las imagenes y si es maligna o benigna en un Dataframe.

In [53]:
# Test
def test_df(test_path):
    classes = [] # Benignas o malignas
    class_paths = [] # Ruta a las imagenes
    image_names = [] # Nombre de las imagenes

    files = os.listdir(test_path) # Archivos
    for file in files: 
        label_dir = os.path.join(test_path, file) # Path del directorio
        label = os.listdir(label_dir) # Imagenes dentro del directorio
        for image in label:
            if not image.startswith('.'):
                image_names.append(image) 
                class_paths.append(label_dir) 
                classes.append(file) # Añade el nombre del archivo

    # Series de pandas
    class_paths = pd.Series(class_paths, name='Class Path')
    image_names = pd.Series(image_names, name='Image name')
    image_classes = pd.Series(classes, name='Class') 

    # Crea el dataframe
    ts_df = pd.concat([class_paths, image_names, image_classes], axis=1)
    return ts_df

ts_df = test_df('./skin-cancer-dataset/test')
print(len(ts_df))

2000


### Normalizar los datos entre 0 y 1
Normalizar los datos de Benigno y Maligno de string a numeros (0 o 1)

In [54]:
tr_df['Class'].replace({'Benign': 0, 'Malignant': 1}, inplace=True)
ts_df['Class'].replace({'Benign': 0, 'Malignant': 1}, inplace=True) 
print(tr_df, ts_df)

                                 Class Path Image name  Class
0     ./skin-cancer-dataset/train/Malignant     63.jpg      1
1     ./skin-cancer-dataset/train/Malignant    823.jpg      1
2     ./skin-cancer-dataset/train/Malignant   1409.jpg      1
3     ./skin-cancer-dataset/train/Malignant    189.jpg      1
4     ./skin-cancer-dataset/train/Malignant     77.jpg      1
...                                     ...        ...    ...
3994     ./skin-cancer-dataset/train/Benign    190.jpg      0
3995     ./skin-cancer-dataset/train/Benign   1404.jpg      0
3996     ./skin-cancer-dataset/train/Benign   1410.jpg      0
3997     ./skin-cancer-dataset/train/Benign    184.jpg      0
3998     ./skin-cancer-dataset/train/Benign   1376.jpg      0

[3999 rows x 3 columns]                                 Class Path Image name  Class
0     ./skin-cancer-dataset/test/Malignant   6400.jpg      1
1     ./skin-cancer-dataset/test/Malignant   6366.jpg      1
2     ./skin-cancer-dataset/test/Malignant   637

## Diferenciar los datos
### Partir las X Y
Indicar al Dataframe train y test cuales son los datos y cuales son los resultados

In [55]:
# Train
trainX = tr_df.values[:, :-1]
trainY = tr_df.values[:, -1]

# Test
testX = ts_df.values[:, :-1]
testY = ts_df.values[:, -1]

### Convertir les Y en tensor
Les X las convertimos despues de obtener la imagen del Path en el Dataframe.

In [56]:
import torch
trainY = torch.tensor(trainY.astype(int))
testY = torch.tensor(testY.astype(int))

## Clase Dataset
Al obtener el dataset devolverá la imagen como tensor, que será la X y los resultados Y.

### **Normalizar las imagenes**:
Las imágenes están contenidas dentro de X en forma de Path por un lado y el image name por otro. Para ello tenemos que convertir los paths a una imagen y normalizar los datos para que sean entre 0 y 1 en vez de 0 a 224.

In [147]:
from torch.utils.data import Dataset
import torchvision.transforms.functional as transform
from torchvision.io import read_image
from pathlib import Path
import PIL
import matplotlib.pyplot as plt

import time

class myDataset(Dataset):
    def __init__(self, X, Y):
        self.image_path = X[:, 0]
        self.image_name = X[:, 1]
        self.Y = Y
        
    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        #print(str(Path(self.image_path[idx]) / self.image_name[idx]))
        #print(read_image(self.image_path[idx]))
        PIL_image = PIL.Image.open(str(Path(self.image_path[idx]) / self.image_name[idx]))
        tensor_image = transform.to_tensor(PIL_image)/255.0 # Convertir a tensor y normalizar
        return tensor_image, self.Y[idx] # Lo devuelves directamente como una imagen

#### Crear el propi dataset
Passar al train_dataloader i test_dataloader un objecte dataset, nosaltres hem de crear aquet dataset extenent de la clase Dataset.

In [163]:
from torch.utils.data import DataLoader

train_dataset = myDataset(trainX, trainY)
test_dataset = myDataset(testX, testY)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

### Crear la red neuronal
1. Crear el dispositivo
2. Definir la clase *Module* con la función forward
    - Convulacional (como es en color, cada imagen se multiplica x 3)
    - Capas normales 
3. Crear el modelo y pasarlo a la GPU

In [149]:
from torch import nn

# Ya están implementadas las clases de las capas para hacer el forward
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            # 3 imatges de 224 x 224
            nn.Conv2d(3, 30, (8, 8)), # Entrada, salida, filtro
            nn.MaxPool2d((4, 4)), 

            nn.Conv2d(30, 120, (5, 5)), # 120 imatges de 69x76
            nn.MaxPool2d((3, 3)), # 120 imatges de 23x25

            #nn.Conv2d(120, 240, (3, 3)), # 250 imatges de 19x21
            #nn.MaxPool2d((3, 3)), # 250 imatges de 6x7
            
            nn.Flatten(),

            nn.Linear(30720, 3000), 
            nn.ReLU(),
            nn.Linear(3000, 300),
            nn.ReLU(),
            nn.Linear(300, 30),
            nn.ReLU(),
            nn.Linear(30, 2),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x) # Crea las capas
        return logits
    
# Device
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

model = NeuralNetwork().to(device) # Otiene los valores predichos
print(device)

mps


### Train y test
Definimos las funciones para train y test.

In [159]:
import torch
from sklearn.metrics import confusion_matrix

batch_size=64

def train_loop(train_dataloader, model, loss_fn, optimizer):
    size = len(train_dataloader.dataset)

    for batch, (X, Y) in enumerate(train_dataloader):
        X = X.to(device) 
        Y = Y.to(device) 
        
        pred = model(X) # Forward, ya ha calculado todos los gradientes
        loss = loss_fn(pred, Y) # Crear la función de costo: error

        loss.backward() # Le pasa el error al gradiente
        optimizer.step() # Actualiza los valores
        optimizer.zero_grad() # Pone el gradiente a 0

        # Accuracy
        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(test_dataloader, model, loss_fn):
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    all_preds = torch.tensor([]).to(device)
    
    test_loss, correct = 0, 0

    # No calcula el gradiente automaticamente
    with torch.no_grad():
        for X, Y in test_dataloader:
            
            X = X.to(device)
            Y = Y.to(device)
            
            pred = model(X) # Forward
            test_loss += loss_fn(pred, Y).item() # Error
            correct += (pred.argmax(1) == Y).type(torch.float).sum().item() # Accuracy

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### Get all predicted values
Of the training set

In [160]:
# Confusion matrix
all_preds = torch.cat((all_preds, pred), dim=0) 

# get all the predictions for the entire training set
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    model.eval() # set model to evaluate mode
    
    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)
        preds = model(images)
        all_preds = torch.cat((all_preds, preds), dim=0)
    
    return all_preds

NameError: name 'all_preds' is not defined

In [161]:
# Confusion matrix

stacked = torch.stack((Y, pred), dim=1) # Concatenar
# Initialize a confusion matrix with 0s
confusion_matrix = torch.zeros(10, 10, dtype=torch.int32)

# fill in the matrix
for row in stacked:
    true_label, pred_label = row.numpy()
    confusion_matrix[true_label, pred_label] += 1

NameError: name 'Y' is not defined

### Llamar a train y test
En cada epoca hacer un train y un test

In [162]:
learning_rate = 0.0001
loss_fn = nn.CrossEntropyLoss()
model = NeuralNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.710129  [   64/ 3999]
Test Error: 
 Accuracy: 50.0%, Avg loss: 0.693242 

Epoch 2
-------------------------------
loss: 0.689658  [   64/ 3999]
Test Error: 
 Accuracy: 50.0%, Avg loss: 0.693461 

Epoch 3
-------------------------------
loss: 0.691251  [   64/ 3999]
Test Error: 
 Accuracy: 70.2%, Avg loss: 0.688960 

Epoch 4
-------------------------------
loss: 0.689058  [   64/ 3999]
Test Error: 
 Accuracy: 70.3%, Avg loss: 0.640091 

Epoch 5
-------------------------------
loss: 0.604699  [   64/ 3999]
Test Error: 
 Accuracy: 68.3%, Avg loss: 0.584959 

Epoch 6
-------------------------------
loss: 0.618053  [   64/ 3999]
Test Error: 
 Accuracy: 71.7%, Avg loss: 0.623358 

Epoch 7
-------------------------------
loss: 0.645988  [   64/ 3999]
Test Error: 
 Accuracy: 73.0%, Avg loss: 0.543013 

Epoch 8
-------------------------------
loss: 0.509122  [   64/ 3999]
Test Error: 
 Accuracy: 68.5%, Avg loss: 0.569907 

Epoch 9
----------------

In [ ]:
img = read_image("/Users/cynthia/Developer/Git/Formación/Dev Git ITIC/Artificial Inteligence/Neural Network/Skin-cancer/skin-cancer-dataset/train/Malignant/63.jpg")

AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'